In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk("/kaggle/input"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Importing Necessary Libraries**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import plotly.express as px

plt.style.use("seaborn-colorblind")

plt.style.use('seaborn-colorblind') --> By setting the style to "seaborn-colorblind", all subsequent plots created using Matplotlib will have a color palette that is optimized for colorblind viewers.

In [ ]:
# nb_black is used for autoformatting
!pip install nb_black --quiet
%load_ext lab_black

In [ ]:
train_df = pd.read_csv("/kaggle/input/asl-signs/train.csv")

In [ ]:
train_df.head()

# **EDA**

In [ ]:
# number of unique signs
train_df["sign"].value_counts()

In [ ]:
train_df["sign"].value_counts().head(30).sort_values().plot(
    kind="barh", figsize=(8, 6), title="Top 30 signs of train data"
)
plt.xlabel("NO. of training samples")
plt.ylabel("Signs")

In [ ]:
train_df["sign"].value_counts().tail(30).sort_values().plot(
    kind="barh", figsize=(8, 6), title="bottom 30 signs of train data"
)
plt.xlabel("NO. of training samples")
plt.ylabel("Signs")

In [ ]:
train_df.query("sign == 'listen'")

# Analysing Single Parquet file
  **for sign = "listen"**

In [ ]:
p1 = train_df.query("sign == 'listen'")["path"].iloc[0]

In [ ]:
p1

In [ ]:
root_dir = "/kaggle/input/asl-signs/"

In [ ]:
p1_file = pd.read_parquet(root_dir + p1)

In [ ]:
p1_file

In [ ]:
frames = p1_file["frame"]
types = p1_file["type"]

print("frame:\n", frames.value_counts())
print(f"this file has {frames.nunique()} unique frames \n")
print("type:\n", types.value_counts())
print(f"this file has {types.nunique()} unique types")

# Comparing parquet files to check what type of data they have

In [ ]:
listen_files = train_df.query("sign == 'listen'")["path"].values

In [ ]:
for i, j in enumerate(listen_files):
    parquet_file = pd.read_parquet(root_dir + j)
    p_frames = parquet_file["frame"]
    p_types = parquet_file["type"]
    print(
        f"this file has {p_frames.nunique()} unique frames and {p_types.nunique()} unique types \n"
    )
    if i == 20:
        break

almost all files has same types but each file hase different number of unique frames

# Create MetaData for Training dataset

"Metadata" is data that provides information about other data, but not the content of the data

In [ ]:
p1_file["type"].value_counts()

In [ ]:
p1_file.dropna(subset=["x", "y", "z"])["type"].value_counts()

In [ ]:
metadata = {}
for i, d in tqdm(train_df.iterrows(), total=len(train_df)):
    file_path = d["path"]
    parquet_file = pd.read_parquet(root_dir + file_path)
    # get the number of landmarks with x,y,z data per type.
    # i.e x,y,z values can be null so we are taking the data with x,y,z having non null values
    meta = parquet_file.dropna(subset=["x", "y", "z"])["type"].value_counts().to_dict()
    meta["frames"] = parquet_file["frame"].nunique()
    xyz = (
        (
            parquet_file[["x", "y", "z"]].agg(
                {
                    "x": ["min", "max", "mean"],
                    "y": ["min", "max", "mean"],
                    "z": ["min", "max", "mean"],
                }
            )
        )
        .unstack()
        .to_dict()
    )

    for k in xyz.keys():
        new_key = k[0] + "_" + k[1]
        meta[new_key] = xyz[k]

    metadata[file_path] = meta

    if i > 1000:
        break

In [ ]:
metadata_df = pd.DataFrame(metadata).T.reset_index().rename(columns={"index": "path"})

In [ ]:
metadata_df

In [ ]:
train_with_meta = train_df.merge(metadata_df, how="left")

In [ ]:
train_with_meta

In [ ]:
train_with_meta.to_parquet("train_with_meta.parquet")

# Finding most frequent types of landmarks provided

In [ ]:
train_with_meta[["face", "pose", "left_hand", "right_hand"]].sum().sort_values().plot(
    kind="barh", title="Sum of Rows by Landmark Type"
)

* Face has a lot more datapoints because mediapipe provides 468 3D datapoints per frame.

In [ ]:
(
    train_with_meta.query("index <= 1001").fillna(0)[
        ["face", "pose", "right_hand", "left_hand"]
    ]
    > 0
).mean().plot(kind="barh", title="Rate of Frame/Keypoints with Data")

Every parquet file has at least some datapoints for all four types of landmarks
* face
* pose
* right_hand
* left_hand

In [ ]:
(
    train_with_meta.query("index < 1000").fillna(0)[
        ["face", "pose", "left_hand", "right_hand"]
    ]
    > 0
).mean().plot(kind="barh", title="Rate of Frame/Keypoints with Data")

# Plotting datapoints for single example

**Taking the datapoints from middle frame of the example**

In [ ]:
example_path = train_with_meta.dropna().query("sign == 'shhh'")["path"].values[0]
example_file = pd.read_parquet(root_dir + example_path)

In [ ]:
example_file["frame"].mean()  # finding the middle frame no. for this example

Taking 25 as middle frame

In [ ]:
# middle frame
example_file.query("frame== 25")["type"].value_counts()

In [ ]:
frame_file = example_file.query("frame== 25")
fig = px.scatter_3d(frame_file, x="x", y="y", z="z", color="type")
fig.show()

In [ ]:
frame_file = example_file.query("frame== 36")
fig = px.scatter_3d(frame_file, x="x", y="y", z="z", color="type")
fig.show()

In [ ]:
example_file["y_"] = example_file["y"] * -1
example_file = example_file.query("frame == 17 and type == 'face'")
px.scatter(example_file, x="x", y="y_", color="type")

# Using mediaPipe for ploting

**taking image from google**

In [ ]:
!wget https://media-cldnry.s-nbcnews.com/image/upload/streams/2012/December/121214/1C5179134-121213-sittingTest-909p.jpg --quiet
!wget https://previews.123rf.com/images/josemagon/josemagon1507/josemagon150701039/42451732-young-man-hands-to-front-on-a-white-background.jpg --quiet
!pip install mediapipe --quiet

In [ ]:
import cv2
import mediapipe as mp

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(6, 8))

axes[0].imshow(
    cv2.cvtColor(cv2.imread("1C5179134-121213-sittingTest-909p.jpg"), cv2.COLOR_BGR2RGB)
)
axes[1].imshow(
    cv2.cvtColor(
        cv2.imread("42451732-young-man-hands-to-front-on-a-white-background.jpg"),
        cv2.COLOR_BGR2RGB,
    )
)

**MediaPipe Holistic Model for Static Images**

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic
mp_hands = mp.solutions.hands

In [ ]:
# For static images:
IMAGE_FILES = [
    "1C5179134-121213-sittingTest-909p.jpg",
    "42451732-young-man-hands-to-front-on-a-white-background.jpg",
]
BG_COLOR = (192, 192, 192)  # gray
with mp_holistic.Holistic(
    static_image_mode=True,
    model_complexity=2,
    enable_segmentation=True,
    refine_face_landmarks=True,
) as holistic:
    for idx, file in enumerate(IMAGE_FILES):
        image = cv2.imread(file)
        image_height, image_width, _ = image.shape
        # Convert the BGR image to RGB before processing.
        results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        if results.pose_landmarks:
            print(
                f"Nose coordinates: ("
                f"{results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].x * image_width}, "
                f"{results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].y * image_height})"
            )

        annotated_image = image.copy()
        # Draw segmentation on the image.
        # To improve segmentation around boundaries, consider applying a joint
        # bilateral filter to "results.segmentation_mask" with "image".
        condition = np.stack((results.segmentation_mask,) * 3, axis=-1) > 0.1
        bg_image = np.zeros(image.shape, dtype=np.uint8)
        bg_image[:] = BG_COLOR
        annotated_image = np.where(condition, annotated_image, bg_image)
        # Draw pose, left and right hands, and face landmarks on the image.
        mp_drawing.draw_landmarks(
            annotated_image,
            results.face_landmarks,
            mp_holistic.FACEMESH_TESSELATION,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style(),
        )
        mp_drawing.draw_landmarks(
            annotated_image,
            results.pose_landmarks,
            mp_holistic.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style(),
        )
        mp_drawing.draw_landmarks(
            annotated_image,
            results.left_hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style(),
        )
        mp_drawing.draw_landmarks(
            annotated_image,
            results.right_hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style(),
        )
        cv2.imwrite("/tmp/annotated_image" + str(idx) + ".png", annotated_image)
        # Plot pose world landmarks.
        mp_drawing.plot_landmarks(
            results.pose_world_landmarks, mp_holistic.POSE_CONNECTIONS
        )

In [ ]:
plt.imshow(plt.imread("/tmp/annotated_image" + str(0) + ".png"))
plt.show()
plt.imshow(plt.imread("/tmp/annotated_image" + str(1) + ".png"))
plt.show()

## Applying same method on single parquet file for plotting

In [ ]:
# File having sign='shhh'
random_example_file_path = train_with_meta.query("sign=='listen'")["path"].iloc[0]
random_example_file = pd.read_parquet(root_dir + random_example_file_path)

### checking for the frame having all types i.e
* face
* pose
* left_hand
* right_hand

In [ ]:
random_example_file.dropna().groupby("frame")["type"].value_counts()

In [ ]:
f1 = random_example_file.dropna().query("frame==40")

In [ ]:
f1["type"].value_counts()

**frame 17 has landmarks for all type**

### **Creating the NormalizedLandmarkList for each type**

In [ ]:
import pandas as pd
from mediapipe.framework.formats import landmark_pb2

landmark_list_face = landmark_pb2.NormalizedLandmarkList()
landmark_list_pose = landmark_pb2.NormalizedLandmarkList()
landmark_list_left_hand = landmark_pb2.NormalizedLandmarkList()
landmark_list_right_hand = landmark_pb2.NormalizedLandmarkList()

for i, row in f1.iterrows():
    if row["type"] == "face":
        landmark = landmark_list_face.landmark.add()
        landmark.x = row["x"]
        landmark.y = row["y"]
        landmark.z = row["z"]
    elif row["type"] == "pose":
        landmark = landmark_list_pose.landmark.add()
        landmark.x = row["x"]
        landmark.y = row["y"]
        landmark.z = row["z"]
    elif row["type"] == "left_hand":
        landmark = landmark_list_left_hand.landmark.add()
        landmark.x = row["x"]
        landmark.y = row["y"]
        landmark.z = row["z"]
    elif row["type"] == "right_hand":
        landmark = landmark_list_right_hand.landmark.add()
        landmark.x = row["x"]
        landmark.y = row["y"]
        landmark.z = row["z"]

In [ ]:
type(landmark_list_left_hand)

In [ ]:
annotated_image.shape

In [ ]:
bg_img = np.zeros([1300, 1146, 3])

In [ ]:
plt.imshow(bg_img)

In [ ]:
mp_drawing.draw_landmarks(
    bg_img,
    landmark_list_face,
    mp_holistic.FACEMESH_TESSELATION,
    landmark_drawing_spec=None,
    connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style(),
)
mp_drawing.draw_landmarks(
    bg_img,
    landmark_list_pose,
    mp_holistic.POSE_CONNECTIONS,
    landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style(),
)

# mp_drawing.draw_landmarks(
#     bg_img,
#     landmark_list_left_hand,
#     mp_hands.HAND_CONNECTIONS,
#     mp_drawing_styles.get_default_hand_landmarks_style(),
#     mp_drawing_styles.get_default_hand_connections_style(),
# )
mp_drawing.draw_landmarks(
    bg_img,
    landmark_list_right_hand,
    mp_hands.HAND_CONNECTIONS,
    mp_drawing_styles.get_default_hand_landmarks_style(),
    mp_drawing_styles.get_default_hand_connections_style(),
)

In [ ]:
plt.imshow(bg_img)
plt.title("Frame 17 landmarks")